In [1]:
import pandas as pd
import numpy as np
from time import time
import logging
import matplotlib.pyplot as plt
import pylab as pl
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import cv2

In [2]:
x=[]
y=[]

In [5]:
import os
# Specify the directory path
# ensure to chnage the label number and the directory name in the below line
directory = r'D:\Pytorch\ML_lab\final_project\Photos\Person1'

# Loop through each file in the directory
for filename in os.listdir(directory):
    # Check if the current item is a file
    if os.path.isfile(os.path.join(directory, filename)):
        # Do something with the file
        #print(os.path.join(directory, filename))
        img=cv2.imread(os.path.join(directory, filename),0)
        # img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(64,48))
        img=np.array(img).flatten()
        x.append(img)
        # change the label number for each class
        y.append(2)

In [6]:
x=np.array(x)
y=np.array(y)

((211,), (211, 3072))

In [7]:
# write all your target names
target_names=['Aditya','Shubh','Jay']

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler as sc
sc=sc()
X_train_scaled= sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)

# Using SVC

In [11]:
## Grid search gives the best possibility
param_grid = { 'C': [1e3, 5e3, 1e4, 5e4, 1e5],
               'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train, Y_train)

print("Best Estimator found by Grid Search:")
print(clf.best_estimator_)


Best Estimator found by Grid Search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0001)


In [12]:
y_pred = clf.predict(X_test)
accuracy_score(y_pred,Y_test)
# print(confusion_matrix(Y_test, y_pred, labels=range(n_classes)))
# print(classification_report(Y_test, y_pred, target_names=target_names))

0.4883720930232558

# Using KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
score=[]
for i in range(1,20):
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,Y_train)
    prediction=knn.predict(X_test)
    score.append(accuracy_score(Y_test,prediction))

In [32]:
knn=KNeighborsClassifier(n_neighbors=12)
knn.fit(X_train,Y_train)
prediction=knn.predict(X_test)
accuracy_score(Y_test,prediction)

0.7906976744186046

In [21]:
prediction=knn.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,prediction)
cm,accuracy_score(Y_test,prediction)

(array([[ 7,  1,  0],
        [ 0, 20,  0],
        [ 1,  5,  9]], dtype=int64),
 0.8372093023255814)

# Using Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=1000,criterion='entropy')
rf.fit(X_train,Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=1000)

In [23]:
prediction=rf.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,prediction)
cm,accuracy_score(Y_test,prediction)

(array([[ 8,  0,  0],
        [ 0, 20,  0],
        [ 1,  2, 12]], dtype=int64),
 0.9302325581395349)

# Save the model using pickle

In [33]:
pick=open('FINAL.pkl','wb')
pickle.dump(knn,pick)
pick.close()

# Load the model using pickle

In [37]:
pick=open('FINAL.pkl','rb')
model_1=pickle.load(pick)
pick.close()
import cv2
import time

In [ ]:


# Load the pre-trained KNN model
knn = model_1
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

target_names=['Aditya','Shubh','Jay']
# Function to recognize faces and put text on the frame
def recognize_faces_and_put_text(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize to the same size as training data
    gray_resized = cv2.resize(gray, (64, 48))
    #faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Flatten the image into a single row
    face_roi = gray_resized.flatten().reshape(1, -1)

    # Predict using the KNN model
   # _, result, _, _ = knn_model.findNearest(face_roi.astype('float32'), k=5)
    prediction=knn.predict(face_roi)
    # Put the predicted label on the frame
    cv2.putText(frame, target_names[int(prediction)], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # for (x, y, w, h) in faces:
    #     cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    return frame

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Call the function to recognize faces and put text
    frame = recognize_faces_and_put_text(frame)

    # Display the frame
    cv2.imshow('Face Recognition', frame)

    # Exit on 'q' press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


C:\Users\shubh\AppData\Local\Temp\ipykernel_29080\3179131865.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, target_names[int(prediction)], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
C:\Users\shubh\AppData\Local\Temp\ipykernel_29080\3179131865.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, target_names[int(prediction)], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
C:\Users\shubh\AppData\Local\Temp\ipykernel_29080\3179131865.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single el